<a href="https://colab.research.google.com/github/Ritika-Jain-18/Ritika_Jain_21223/blob/main/QUE4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


#Problem Statement: Assign every adjacent state of Australia a different color (Graph coloring problem)

# To model the problem as a CSP, we need to define the variables, domains, and constraints.
# The variables are the seven regions of Australia (at least the seven that we’ll restrict ourselves to): Western Australia; Northern Territory; South Australia; Queensland; New South Wales; Victoria; and Tasmania. In our CSP, they can be modeled with strings.
# The domain of each variable is the three different colors that can possibly be assigned (we’ll use red, green, and blue).
# The constraints are the tricky part. No two adjacent regions can be colored with the same color, and our constraints are dependent on which regions border one another.
# We can use binary constraints (constraints between two variables). Every two regions that share a border also share a binary constraint indicating they can’t be assigned the same color.

# To implement these binary constraints in code, we need to subclass the Constraint class.
# The MapColoringConstraint subclass takes two variables in its constructor (therefore being a binary constraint): the two regions that share a border.
# Its overridden satisfied() method check whether the two regions both have a domain value (color) assigned to them—if either doesn’t, the constraint’s trivially satisfied until they do (there can’t be a conflict when one doesn’t yet have a color).
# Then it checks whether the two regions are assigned the same color (obviously there’s a conflict, meaning the constraint isn’t satisfied, when they’re the same).



/bin/bash: -c: line 1: syntax error near unexpected token `/content/Kopec_CSPiP_02.png'
/bin/bash: -c: line 1: `[picture] (/content/Kopec_CSPiP_02.png)'


In [ ]:
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod
V = TypeVar('V') # variable type
D = TypeVar('D') # domain type

##Constraints are defined using a Constraint class. Each Constraint consists of the variables it constrains and a method that checks whether it’s satisfied().
 # Base class for all constraints
class Constraint(Generic[V, D], ABC):
     # The variables that the constraint is between
     def __init__(self, variables: List[V]) -> None:
         self.variables = variables

     # Must be overridden by subclasses
     @abstractmethod
     def satisfied(self, assignment: Dict[V, D]) -> bool:
      ...

In [ ]:
# A constraint satisfaction problem consists of variables of type V
 # that have ranges of values known as domains of type D and constraints
 # that determine whether a particular variable's domain selection is valid
class CSP(Generic[V, D]):
 ## The __init__() initializer creates the constraints dict.
  ##The add_constraint() method goes through all of the variables touched by a given constraint and adds itself to the constraints
 ## mapping for each of them. Both methods have basic error-checking in place, and raise an exception when a variable is missing a domain or a constraint is on a nonexistent variable.
     def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
         self.variables: List[V] = variables # variables to be constrained
         self.domains: Dict[V, List[D]] = domains # domain of each variable
         self.constraints: Dict[V, List[Constraint[V, D]]] = {}
         for variable in self.variables:
             self.constraints[variable] = []
             if variable not in self.domains:
                 raise LookupError("Every variable should have a domain assigned to it.")

     def add_constraint(self, constraint: Constraint[V, D]) -> None:
         for variable in constraint.variables:
             if variable not in self.variables:
                 raise LookupError("Variable in constraint not in CSP")
             else:
                 self.constraints[variable].append(constraint)
    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
   # consistent() goes through every constraint for a given variable
   #  (it’s always a variable that was newly added to the assignment) and checks if the constraint is satisfied, given the new assignment.
   # If the assignment satisfies every constraint, True is returned. If any constraint imposed on the variable isn’t satisfied, False is returned.
     def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
         for constraint in self.constraints[variable]:
           if not constraint.satisfied(assignment):
               return False
         return True
    #The backtracking search implemented in the following backtracking_search() function is a kind of recursive depth-first search.
     def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
     # assignment is complete if every variable is assigned (our base case) The base case for the recursive search is finding a valid assignment for every variable. Once we have, we return the first instance of a solution that was valid (we stop searching).
         if len(assignment) == len(self.variables):
            return assignment
     # get all variables in the CSP but not in the assignment
         unassigned: List[V] = [v for v in self.variables if v not in assignment]

     # get the every possible domain value of the first unassigned variable
         first: V = unassigned[0]

         #To select a new variable whose domain we can explore, we go through all of the variables and find the first that doesn’t have an assignment.
         #To do this, we create a list of variables in self.variables but not in assignment through a list comprehension, and call it unassigned.
         #Then we pull out the first value in unassigned.
         for value in self.domains[first]:
           local_assignment = assignment.copy()
           local_assignment[first] = value
         # if we're still consistent, we recurse (continue)
         #We try assigning every possible domain value for that variable, one at a time. The new assignment for each is stored in a local dictionary called local_assignment.
           if self.consistent(first, local_assignment):
             result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
             # if we didn't find the result, we will end up backtracking
             if result is not None:
                 return result
         return None

In [ ]:
from typing import Dict, List, Optional


class MapColoringConstraint(Constraint[str, str]):
     def __init__(self, place1: str, place2: str) -> None:
         super().__init__([place1, place2])
         self.place1: str = place1
         self.place2: str = place2

     def satisfied(self, assignment: Dict[str, str]) -> bool:
         # If either place is not in the assignment then it is not
         # yet possible for their colors to be conflicting
         if self.place1 not in assignment or self.place2 not in assignment:
             return True
         # check the color assigned to place1 is not the same as the
         # color assigned to place2
         return assignment[self.place1] != assignment[self.place2]

In [ ]:
#Now that we have a way of implementing the constraints between regions, fleshing out the Australian map-coloring problem
#with our CSP solver is a matter of filling in domains and variables, and then adding constraints.

if __name__ == "__main__":
     variables: List[str] = ["Western Australia", "Northern Territory", "South Australia",
                             "Queensland", "New South Wales", "Victoria", "Tasmania"]
     domains: Dict[str, List[str]] = {}
     for variable in variables:
         domains[variable] = ["red", "green", "blue"]
     csp: CSP[str, str] = CSP(variables, domains)
     csp.add_constraint(MapColoringConstraint("Western Australia", "Northern Territory"))
     csp.add_constraint(MapColoringConstraint("Western Australia", "South Australia"))
     csp.add_constraint(MapColoringConstraint("South Australia", "Northern Territory"))
     csp.add_constraint(MapColoringConstraint("Queensland", "Northern Territory"))
     csp.add_constraint(MapColoringConstraint("Queensland", "South Australia"))
     csp.add_constraint(MapColoringConstraint("Queensland", "New South Wales"))
     csp.add_constraint(MapColoringConstraint("New South Wales", "South Australia"))
     csp.add_constraint(MapColoringConstraint("Victoria", "South Australia"))
     csp.add_constraint(MapColoringConstraint("Victoria", "New South Wales"))
     csp.add_constraint(MapColoringConstraint("Victoria", "Tasmania"))
     #Backtracking_search() is called to find a solution.
     solution:Optional[Dict[str, str]] = csp.backtracking_search()

     if solution is None:
        print ("test",solution)
        print("No solution found!")
     print(solution)
#Content taken from https://freecontent.manning.com/constraint-satisfaction-problems-in-python/

{'Western Australia': 'red', 'Northern Territory': 'green', 'South Australia': 'blue', 'Queensland': 'red', 'New South Wales': 'green', 'Victoria': 'red', 'Tasmania': 'green'}
